# GenAI for Insurance: Multi-Document Summarization Demo


## 1. Introduction

This notebook demonstrates how to apply Multi-Document Summarization (MDS) techniques to insurance product documentation.

We will use a pre-processed dataset of insurance documents and a summarization pipeline based on LangChain APIs and ChatGPT-4o Mini.


## 2. Setup and Imports

In [ ]:
# Install required libraries (if not already installed)


In [ ]:
# Import libraries
import os
from PyPDF2 import PdfReader, PdfWriter
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document


In [ ]:
# Load OpenAI API key
# Instructions: https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key
open_api_key = "Insert you OpenAI API key here"
os.environ["OPENAI_API_KEY"]=open_api_key

## 3. Document Split

In [ ]:
# Pre-defined sections for the selected document
sections =[(3, 4), (7, 32), (33, 34), (39, 49), (50, 60), (61, 79), (80, 110), (111, 135), (136, 143), (144, 150), (151, 172), (173, 202), (204, 209)]

# Create files from intervals
pdf_reader = PdfReader("1.pdf")
os.makedirs("documents")
for i, interval in enumerate(sections):
    start, end = interval

    pdf_writer = PdfWriter()
    for page_num in range(start - 1, end):
        pdf_writer.add_page(pdf_reader.pages[page_num])
    # Create the output file
    output_file = f"documents/{i+1}.pdf"
    with open(output_file, "wb") as output_pdf:
        pdf_writer.write(output_pdf)

    print(f"Created file: {output_file}")


## 4. Prompt definition

In [ ]:
# Map prompt with CoE technique for insurance documents
map_template = "\nOra sei un generatore di riassunti multi-documento. Il tuo <Input> \u00e8 un insieme di testi assicurativi che riguardano un prodotto assicurativo specifico. Dovrai estrarre le informazioni principali e specifiche relative a ciascun documento, concentrandoti su dettagli finanziari e di copertura.\n\n<Input>\n{Input}\n</Input>\n\n<Estrazione delle informazioni critiche>\nPer ciascun documento nel <Input>, estrai e riassumi le informazioni critiche relative al prodotto assicurativo. In particolare, concentrati sui seguenti aspetti chiave e produci una singola linea per ciascun documento, includendo le informazioni finanziarie e di copertura.\n\n1. Descrizione del prodotto e tipi di copertura (es. decesso, invalidit\u00e0 permanente, danni materiali, ecc.)\n2. Limiti di copertura e importi di rimborso specifici (es. \u20ac600 per incidente, \u20ac1.000 per danni elettrici).\n3. Esclusioni (cosa non \u00e8 coperto dalla polizza).\n4. Franchigie e altri dettagli finanziari (es. \u20ac150 di franchigia per danni da acqua).\n5. Opzioni di copertura aggiuntiva, inclusi eventuali costi aggiuntivi.\n6. Durata del contratto, termini di rinnovo e pagamento.\n7. Processo di gestione dei sinistri e condizioni per il rimborso.\n\nQuesto \u00e8 un esempio di estrazione delle informazioni critiche:\n<Esempio di estrazione delle informazioni critiche>\nTesto 1: Il prodotto \"VITA SICURA\" offre una copertura per decesso fino a \u20ac500.000; copertura in caso di invalidit\u00e0 permanente fino a \u20ac250.000; esclusioni includono malattie preesistenti e sport pericolosi; franchigia del 5% per invalidit\u00e0; durata del contratto: 10 anni con rinnovo automatico.\nTesto 2: Il prodotto \"CASA PROTETTA\" offre copertura per danni materiali fino a \u20ac300.000; franchigia di \u20ac400 per sinistro; esclusione per danni derivanti da negligenza; opzioni aggiuntive per copertura danni da acqua a \u20ac200/anno.\n</Esempio di estrazione delle informazioni critiche>\n\nSeguendo l'esempio sopra, estrai le informazioni critiche per ciascun documento assicurativo del <Input>, producendo una singola riga per ciascun documento.\n"
print(map_template)

In [ ]:
# Reduce prompt with CoE technique for insurance documents
reduce_template = "\nIl tuo <Input> \u00e8 un insieme di riassunti specifici estratti da documenti assicurativi. Il tuo compito \u00e8 analizzare questi riassunti, astrarre le informazioni principali e combinarle in una sintesi coesa.\n\n<Input>\n{Input}\n</Input>\n\n<Astrazione delle informazioni e generalizzazione>\n1. Analizza ciascun riassunto estratto nella fase MAP e astrai le caratteristiche comuni del prodotto assicurativo.\n2. Generalizza i dettagli chiave relativi a:\n - Tipi di copertura e limiti di rimborso.\n - Esclusioni principali.\n - Franchigie e importi finanziari specifici.\n - Durata del contratto e condizioni di rinnovo.\n3. Raccogli i dettagli specifici e condensali in una singola versione astratta del prodotto, omettendo dettagli superflui o ridondanti.\n\nQuesto \u00e8 un esempio di astrazione e generalizzazione:\n<Esempio di astrazione e generalizzazione>\nInput:\nRiassunto 1: Il prodotto \"VITA SICURA\" offre una copertura per decesso fino a \u20ac500.000, invalidit\u00e0 permanente fino a \u20ac250.000, esclusioni per malattie preesistenti, con franchigia del 5%.\nRiassunto 2: Il prodotto \"CASA PROTETTA\" include copertura per danni materiali fino a \u20ac300.000, con esclusione per negligenza e franchigia di \u20ac400.\n\nRisultato di astrazione e generalizzazione:\n- Copertura del prodotto per eventi critici come decesso e danni materiali con limiti di \u20ac500.000 e \u20ac300.000 rispettivamente.\n- Esclusioni comuni includono malattie preesistenti e negligenza.\n- Franchigie comprese tra il 5% e \u20ac400 a seconda del tipo di sinistro.\n</Esempio di astrazione e generalizzazione>\n\n<Statistica degli elementi comuni>\nDopo aver generalizzato le informazioni, trova gli elementi che ricorrono in pi\u00f9 documenti e seleziona quelli che coprono la maggior parte dei testi. Questi elementi saranno centrali nel riassunto finale.\n\nQuesto \u00e8 un esempio di statistica degli elementi comuni:\n<Esempio di statistica degli elementi comuni>\nElemento comune 1: Copertura per decesso (testi 1, 2, 3)\nElemento comune 2: Esclusioni per malattie preesistenti (testi 1, 2)\nElemento comune 3: Franchigia del 5% per sinistri di invalidit\u00e0 (testi 1, 3)\n</Esempio di statistica degli elementi comuni>\n\n<Generazione del riassunto finale>\nInfine, integra gli elementi selezionati e generalizzati per formare un riassunto finale coeso e ordinato cronologicamente o per importanza.\n\nEsempio di riassunto finale:\nIl prodotto assicurativo \"VITA SICURA PLUS\" fornisce copertura per decesso fino a \u20ac500.000, con protezione aggiuntiva in caso di invalidit\u00e0 fino a \u20ac250.000. Esclusioni principali comprendono malattie preesistenti non dichiarate e sport estremi. Il contratto prevede una franchigia del 5% per sinistri di invalidit\u00e0 e una copertura opzionale per malattie critiche.\n</Esempio di riassunto finale>\n\nAssicurati che il riassunto finale sia scritto in italiano e compreso tra 300 e 500 parole.\n"
print(reduce_template)

## 5. Map-Reduce Summarization

In [ ]:
#Map
map_prompt = PromptTemplate.from_template(map_template)

# Define LLM Map chain
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [ ]:
# Reduce
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Take a list of documents, combine them into a single string, and pass this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="Input"
)

# Combine and iteratively reduce the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # The final chain
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=16000,
)

In [ ]:
# Apply MapReduce chain to all clusters
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="Input",
    # Return the results of the map steps in the output
    return_intermediate_steps=True,
    # Print log messages while generating the summary
    verbose=True,
)

In [ ]:
# Create the Document object to use it in the LLM Chain
path = "./documents/"
docs_split = []
for file in os.listdir(path):
    file_path = path+file
    pages = PyPDFLoader(file_path).load()
    text = ''.join([elem.page_content for elem in pages])
    docs_split.append(Document(page_content=text, metadata={'filename': file}))

In [ ]:
# Compute and print the summary
generated_summaries = []
res = map_reduce_chain.invoke(docs_split)
summary = res["output_text"] #Final summary
int_steps = res["intermediate_steps"] #Intermediate steps

In [ ]:
# Print the result summary
print(summary)

In [ ]:
# Print the intermediate steps
for elem in int_steps:
    print(elem)

## 6. Conclusion

This notebook shows how GenAI can simplify the analysis of complex insurance documentation using Multi-Document Summarization.

👉 For more insights, refer to the full article: [Link to Medium post]
